In [7]:
import pandas as pd
from pandas import DataFrame
from sklearn.preprocessing import MinMaxScaler

folk = pd.read_csv('..\dataset generation\small_folk_features.csv')
folk['Class'] = 'folk'
folk = folk.iloc[2:, 1:]
#print(folk.head())
#print(folk.shape)

instrumental = pd.read_csv('..\dataset generation\small_instrumental_features.csv')
instrumental['Class'] = 'instrumental'
instrumental = instrumental.iloc[2:, 1:]
#print(instrumental.head())
#print(instrumental.shape)

hip_hop = pd.read_csv('..\dataset generation\small_hip_hop_features.csv')
hip_hop['Class'] = 'hip_hop'
hip_hop = hip_hop.iloc[2:, 1:]
#print(hip_hop.head())
#print(hip_hop.shape)

elec = pd.read_csv('..\dataset generation\small_elec_features.csv')
elec['Class'] = 'elec'
elec = elec.iloc[2:, 1:]
#print(elec.head())
#print(elec.shape)

rock = pd.read_csv('..\dataset generation\small_rock_features.csv')
rock['Class'] = 'rock'
rock = rock.iloc[2:, 1:]
#print(rock.head())
#print(rock.shape)

df = pd.concat([folk, instrumental, hip_hop, elec, rock])
df = df.sample(frac=1).reset_index(drop=True)
#print(df.head())
#print(df.shape)

y_data = df['Class']
x_data = df.drop('Class', axis = 1)

class_data = pd.get_dummies(df['Class'], drop_first = False)

scaler = MinMaxScaler()
scaled = scaler.fit_transform(x_data)
x_df = DataFrame(scaled)

df = pd.concat([x_df, class_data], axis = 1)
df = df.dropna()
print(df.head())
print(df.shape)

          0         1         2         3         4         5         6  \
0  0.007605  0.028594  0.012270  0.018464  0.010110  0.011495  0.021499   
1  0.007127  0.028791  0.019331  0.017479  0.005351  0.010375  0.039136   
2  0.008560  0.042266  0.026561  0.019975  0.009371  0.014650  0.023473   
3  0.006370  0.035545  0.026172  0.012570  0.006778  0.006021  0.017927   
4  0.007044  0.027775  0.011996  0.014809  0.005833  0.005513  0.021590   

          7         8         9  ...       513       514       515       516  \
0  0.024051  0.050537  0.008696  ...  0.077271  0.070950  0.096552  0.155645   
1  0.027949  0.062206  0.023098  ...  0.060679  0.041499  0.041379  0.203818   
2  0.032820  0.032919  0.011810  ...  0.233942  0.214190  0.193103  0.182896   
3  0.031094  0.049663  0.011438  ...  0.091518  0.087015  0.034483  0.159479   
4  0.018702  0.023340  0.008422  ...  0.188950  0.170013  0.172414  0.173703   

        517  elec  folk  hip_hop  instrumental  rock  
0  0.046400  

In [8]:
from sklearn.model_selection import train_test_split

y_cols = ['folk', 'instrumental', 'hip_hop', 'rock', 'elec']
y_data = df[y_cols]
x_data = df.drop(y_data, axis = 1)

x_training_data, x_test_data, y_training_data, y_test_data = train_test_split(x_data, y_data, test_size = 0.1)

In [9]:
from sklearn.metrics import classification_report
from sklearn.metrics import multilabel_confusion_matrix
from sklearn.metrics import mean_squared_error
from sklearn.metrics import accuracy_score
from sklearn.neural_network import MLPClassifier

mlp = MLPClassifier(solver = 'sgd', random_state = 42, activation = 'logistic', learning_rate_init = 0.3, batch_size = 300, hidden_layer_sizes = (200,100), max_iter = 500)
mlp.fit(x_training_data, y_training_data)
predictions = mlp.predict(x_test_data)

print("Confusion Matrix:")
print(multilabel_confusion_matrix(y_test_data, predictions))
print("\nAccuracy:", accuracy_score(y_test_data, predictions))
print("\nMSE:", mean_squared_error(y_test_data, predictions))
print("\nClassification Report:")
print(classification_report(y_test_data, predictions))

Confusion Matrix:
[[[358  32]
  [ 27  83]]

 [[386  13]
  [ 63  38]]

 [[390  23]
  [ 25  62]]

 [[380  16]
  [ 30  74]]

 [[387  15]
  [ 47  51]]]

Accuracy: 0.61

MSE: 0.11639999999999999

Classification Report:
              precision    recall  f1-score   support

           0       0.72      0.75      0.74       110
           1       0.75      0.38      0.50       101
           2       0.73      0.71      0.72        87
           3       0.82      0.71      0.76       104
           4       0.77      0.52      0.62        98

   micro avg       0.76      0.62      0.68       500
   macro avg       0.76      0.62      0.67       500
weighted avg       0.76      0.62      0.67       500
 samples avg       0.61      0.62      0.61       500



c:\Users\micha\anaconda3\envs\env1\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [12]:
from sklearn.model_selection import GridSearchCV, KFold

learning_rate_init = [0.2, 0.3, 0.4]
max_iter = [300, 500, 700]
hidden_layer_sizes = [(200,100), (256,64,8), (200,50)]
solver = ['sgd']
random_state = [42]
activation = ['logistic']
batch_size = [300]


# Make a dictionary of the grid search parameters
param_grid = dict(learning_rate_init=learning_rate_init, max_iter=max_iter, hidden_layer_sizes=hidden_layer_sizes, solver=solver, random_state=random_state, activation=activation, batch_size=batch_size)

# Build and fit the GridSearchCV
grid = GridSearchCV(estimator=mlp, param_grid=param_grid,
                    cv=KFold(), verbose=10)

grid_results = grid.fit(x_data, y_data)

Fitting 5 folds for each of 27 candidates, totalling 135 fits
[CV 1/5; 1/27] START activation=logistic, batch_size=300, hidden_layer_sizes=(200, 100), learning_rate_init=0.2, max_iter=300, random_state=42, solver=sgd
[CV 1/5; 1/27] END activation=logistic, batch_size=300, hidden_layer_sizes=(200, 100), learning_rate_init=0.2, max_iter=300, random_state=42, solver=sgd;, score=0.611 total time=   6.7s
[CV 2/5; 1/27] START activation=logistic, batch_size=300, hidden_layer_sizes=(200, 100), learning_rate_init=0.2, max_iter=300, random_state=42, solver=sgd
[CV 2/5; 1/27] END activation=logistic, batch_size=300, hidden_layer_sizes=(200, 100), learning_rate_init=0.2, max_iter=300, random_state=42, solver=sgd;, score=0.634 total time=   6.3s
[CV 3/5; 1/27] START activation=logistic, batch_size=300, hidden_layer_sizes=(200, 100), learning_rate_init=0.2, max_iter=300, random_state=42, solver=sgd
[CV 3/5; 1/27] END activation=logistic, batch_size=300, hidden_layer_sizes=(200, 100), learning_rate_

In [14]:
print("Optimal Paramaters:", grid_results.best_params_)
print("\nOptimal Hidden Layer Sizes:", grid_results.best_params_['hidden_layer_sizes'])
print("\nOptimal Learning Rate:", grid_results.best_params_['learning_rate_init'])
print("\nOptimal Number of Epochs:", grid_results.best_params_['max_iter'])
print("\nMax Accuracy:", grid_results.best_score_)

Optimal Paramaters: {'activation': 'logistic', 'batch_size': 300, 'hidden_layer_sizes': (200, 100), 'learning_rate_init': 0.2, 'max_iter': 300, 'random_state': 42, 'solver': 'sgd'}

Optimal Hidden Layer Sizes: (200, 100)

Optimal Learning Rate: 0.2

Optimal Number of Epochs: 300

Max Accuracy: 0.6254512512512512
